# A. Weight Analysis Project

The Task: Determining the factors that play a role in predicting an indivitual's weight.

In [ ]:
%%html
<img src="pictures/weight.jpg", width=500, height=500>

The data used for this project is taken from the National Health and Nutrition Examination Survey of individuals from the United States of America. The following information was collected from participants in this survey:

1. Weight - in kilograms - Numerical
2. Age - in years - Numerical
3. Height - in centimetres - Numerical
4. Income Ratio: Ratio of Participant Income to baseline poverty level income - Numerical
5. Sex - Male or Female - Categorical
6. Activity - Number of dats in the last week including more than 60 minutes of excercise - Numerical

Investigation: The end point of this project is to build a regressional model to test whether the chosen features effectively predict an indivituals weight or not. In order build up to this, some preliminary data cleaning and analysis needs to be conducted in order to deal with missing data and exclude any covariates that would not be as relevant to the model.

# B. Data Inspection and Cleaning

First we must import the necessary packages for dealing with the datasets:
- pandas
- matplotlib
- seaborn

In [ ]:
#Importing packages:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

The dataset for this project exists as the file excelica.csv. I will load this file and use the .head() to inspect the first 5 rows of the dataset.

In [ ]:
excelica = pd.read_csv("excelica.csv")
excelica.head()

As can be seen, there are columns which have NaN or null values. Let's find out how many rows are in this dataset and then identify the number of null values in each column specifically.

In [ ]:
#Find total number of rows in dataframe
print("Number of rows in dataframe : " + str(len(excelica)))
#Find the number of NaN values in each column of the dataframe
print("\nNumber of null values in dataframe: \n" + str(excelica.isna().sum()))

As can be seen above, activity has the greatest number of null values(7086 values!), followed by height and income(rougly 1000 values). This is problematic as pandas will perform a list-wise deletion during regression: if a record has atleast 1 null value, that entire record will be excluded from the development of the regression line. So we need to deal with this missing data.

# C. Preliminary Analysis

The aim of doing this analysis is to check for how spread out the dataset is and if there is any prevalent bias in the population. When we are determining if the predictors mentioned in section A have any relationship to the Weight, we need to see that the participants are not of one particular type (ex. mostly tall men, or older women etc). Biases such as these can limit how well our Linear Regression model will predict for new participants.

First, lets obtain summary statistics for the continuous variables of this dataframe. We can use .describe() for this.

In [ ]:
#Obtaining summary stats for continuous variables
print(excelica.describe())

#Number of males and females
males = excelica[excelica.Sex == "Male"]
females = excelica[excelica.Sex == "Female"]
print("\nNumber of males: " + str(len(males)))
print("Number of females: " + str(len(females)))

The number of males and females involved in this survey are not equal, although they are roughly around same range so we could still consider analysis to not be too biased. Now lets check the distributions of the continuous variables.

In [ ]:
#Isolate each of the continuous variables
weight = excelica["Weight"]
activity = excelica["Activity"]
age = excelica["Age"]
height = excelica["Height"]
income = excelica["Income"]
sex = excelica["Sex"]

In [ ]:
#Histogram of weights distribution
plt.figure()
plt.hist(weight, bins=50, histtype="bar", density=True, color="orange")
plt.xlabel("Weights")
plt.ylabel("Frequency Normed")
plt.title("Distribution of Weights of Survey responders")
plt.show()

#Histogram of ages distribution
plt.figure()
plt.hist(age, histtype="bar", density=True, color="blue")
plt.xlabel("Age")
plt.ylabel("Frequency Normed")
plt.title("Distribution of Ages of Survey responders")
plt.show()

#Histogram of heights distribution
plt.figure()
plt.hist(height, bins=20, histtype="bar", density=True)
plt.xlabel("Heights")
plt.ylabel("Frequency Normed")
plt.title("Distribution of Heights of Survey responders")
plt.show()

#Histogram of activity distribution
plt.figure()
plt.hist(activity, histtype="bar", bins=7, density=True, color="green")
plt.xlabel("Activity")
plt.ylabel("Frequency Normed")
plt.title("Distribution of Activity of Survey responders")
plt.show()

#Histogram of income distribution
plt.figure()
plt.hist(income, histtype="bar", bins=15, density=True, color="crimson")
plt.xlabel("Income")
plt.ylabel("Frequency Normed")
plt.title("Distribution of Income of Survey responders")
plt.show()

Because the data also has a categorical variable, which is the sex, and this definitely has some biological relation to weight, it is also important to check the distribution of the ages and heights of the sexes. We could use violin plots for studying this. A second histogram is also created, just for comparison to make sure that the written code was correct!

In [ ]:
males_heights = excelica["Height"][excelica["Sex"] == "Male"].dropna()
females_heights = excelica["Height"][excelica["Sex"] == "Female"].dropna()

#Plotting a violinplot of sex vs height
plt.figure()
plt.title("Comparison of Height Distribution by Sex: ViolinPlot")
sns.violinplot(x=sex, y=height)
plt.show()

#Plotting a histogram of heights of males and females
plt.figure()
plt.title("Comparison of Height Distribution by Sex: Histogram")
plt.hist(males_heights, bins=50, density=True, alpha=0.5)
plt.hist(females_heights, bins=50, density=True, alpha=0.5)
plt.xlabel("Heights")
plt.ylabel("Frequency Normed")
plt.legend(["Male", "Female"])
plt.show()

In [ ]:
males_weights = excelica["Weight"][excelica["Sex"] == "Male"].dropna()
females_weights = excelica["Weight"][excelica["Sex"] == "Female"].dropna()

#Plotting a histogram of heights of males and females
plt.figure()
plt.title("Comparison of Weight Distribution by Sex: Histogram")
plt.hist(males_weights, bins=50, density=True, alpha=0.5)
plt.hist(females_weights, bins=50, density=True, alpha=0.5)
plt.xlabel("Weights")
plt.ylabel("Frequency Normed")
plt.legend(["Male", "Female"])
plt.show()

#print(males_weights.mean())
#print(females_weights.mean())
print(males_weights.var())
print(females_weights.var())

Looking at this data, we can see that the height distribution is fairly similar between males and females.The mean height of males appears to be greater than females. We can check if there is a significant difference between the mean male and female heights using an ANOVA test.

In [ ]:
#Importing ANOVA test from scipy.stats module
from scipy.stats import f_oneway

In order to do an ANOVA, it was required that the rows of male and female heights that stored null values be dropped.

In [ ]:
#Performing ANOVA to compare mean heights of males and females
fstat, pval = f_oneway(males_heights, females_heights)
print("H0: µ of male heights = µ of female heights")
print("H1: µ of male heights != µ of female heights")
print("p-value: " + str(pval))

The p-value for the ANOVA comparing males and females heights is lesser than 0.05. Thus, we can reject the H0 and conclude that the males and females heights are different.

In [ ]:
males_ages = excelica["Age"][excelica["Sex"] == "Male"]
females_ages = excelica["Age"][excelica["Sex"] == "Female"]

#Plotting a violinplot of sex vs height
plt.figure()
plt.title("Comparison of Age Distribution by Sex: ViolinPlot")
sns.violinplot(x=sex, y=age)
plt.show()

#Plotting a histogram of heights of males and females
plt.figure()
plt.title("Comparison of Height Distribution by Sex: Histogram")
plt.hist(males_ages, bins=10, density=True, histtype="step",alpha=1)
plt.hist(females_ages, bins=10, density=True, histtype="step", alpha=1)
plt.xlabel("Age")
plt.ylabel("Frequency Normed")
plt.legend(["Male", "Female"])
plt.show()

For this data as well, the ages of males and females involved in this survey are very similar. We can conduct another ANOVA to compare the mean age of the male and female populations.

In [ ]:
#Performing ANOVA to compare mean age of males and females
fstat, pval = f_oneway(males_ages, females_ages)
print("H0: µ of male ages = µ of female ages")
print("H1: µ of male ages != µ of female ages")
print("p-value: " + str(pval))

The p-value is greater than 0.05, hence we do not reject the null hypothesis and thus conclude that the male and female mean ages are similar.

In [ ]:
#Performing ANOVA to compare mean weights of males and females
fstat, pval = f_oneway(males_weights, females_weights)
print("H0: µ of male weights = µ of female weights")
print("H1: µ of male weights != µ of female weights")
print("p-value: " + str(pval))

In [ ]:
def scatterplot_maker(rows, columns, X, X_labels, y, y_label):
    fig = plt.figure(figsize=[20,6])
    plt.rcParams.update({'font.size':20})
    for i in range(columns):
        ax = plt.subplot(rows, columns, i+1)
        plt.scatter(X[i], y, edgecolors='black', linewidth=1)
        plt.xlabel(X_labels[i])
        plt.ylabel(y_label)
        text = 'r = ' + str(X[i].corr(y))
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, text, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
        plt.savefig('corr2.png')
    plt.show()

X = [age, height]
X_labels = ['Age', 'Height']
y = weight
y_label = 'Weight'
scatterplot_maker(rows=1, columns=2, X=X, y=y, X_labels=X_labels, y_label=y_label)

plt.figure(figsize=[6, 6])
ax = plt.subplot()
plt.scatter(income, weight, edgecolors='black', linewidth=1)
plt.xlabel('Income')
plt.ylabel('Weight')
text = 'r = ' + str(income.corr(weight))
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
ax.text(0.05, 0.95, text, transform=ax.transAxes, fontsize=14,
verticalalignment='top', bbox=props)
plt.savefig('corr_income.png')
plt.show()

# Linear Regression Analysis

In [ ]:
excelica_cleaned = excelica.drop(columns = "Activity")
excelica_cleaned.head()
excelica_cleaned.dropna()
excelica_cleaned.isna().any()

In [ ]:
features = ["Age", "Height", "Income"]
X = excelica[features]
y = excelica["Weight"]
X.isna().any()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
weight_predictor = LinearRegression()
weight_predictor.fit(x_train, y_train)
weight_predictor.coef_